In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
import nltk
from nltk import pos_tag
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords

df = pd.read_csv('/content/drive/Shareddrives/Text mining Project/PROJECT/TWEET 2.csv')

In [ ]:
!wget http://nlp.stanford.edu/data/glove.twitter.27B.zip
!unzip glove.twitter.27B.zip

--2023-12-05 07:01:01--  http://nlp.stanford.edu/data/glove.twitter.27B.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://nlp.stanford.edu/data/glove.twitter.27B.zip [following]
--2023-12-05 07:01:02--  https://nlp.stanford.edu/data/glove.twitter.27B.zip
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://downloads.cs.stanford.edu/nlp/data/glove.twitter.27B.zip [following]
--2023-12-05 07:01:02--  https://downloads.cs.stanford.edu/nlp/data/glove.twitter.27B.zip
Resolving downloads.cs.stanford.edu (downloads.cs.stanford.edu)... 171.64.64.22
Connecting to downloads.cs.stanford.edu (downloads.cs.stanford.edu)|171.64.64.22|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1520408563 (1.4G) [ap

In [ ]:
import pandas as pd
import numpy as np
from gensim.models import KeyedVectors
glove_model = KeyedVectors.load_word2vec_format('glove.twitter.27B.200d.txt', binary=False, no_header=True)


# Function to convert a sentence to a vector using GloVe word vectors
def sentence_to_vector(sentence, model):
    words = sentence.split()
    vector = sum([model[word] for word in words if word in model])
    return vector / len(words) if len(words) > 0 else np.zeros(200)  # Adjust dimension based on the loaded model

# Apply the function to the 'text' column in your DataFrame
df['embedding'] = df['text'].apply(lambda x: sentence_to_vector(x, glove_model))


In [ ]:
df['embedding']

0        [-0.012313548, 0.029266087, 0.008681902, 0.299...
1        [-0.052466247, -0.0930375, 0.1549125, 0.065154...
2        [0.02908667, -0.16832334, 0.22968334, 0.175442...
3        [-0.040377542, 0.15304908, 0.09444019, 0.23846...
4        [0.21581136, 0.03897329, 0.17565462, -0.003136...
                               ...                        
40618    [0.026347542, 0.15136111, 0.021407995, -0.1230...
40619    [0.1294142, 0.043254435, 0.00953022, -0.117486...
40620    [-0.029319227, -0.035034575, 0.24909759, 0.282...
40621    [-0.010016665, -0.17406034, -0.029616654, 0.34...
40622    [-0.08816388, 0.073139995, 0.0424425, 0.224486...
Name: embedding, Length: 40623, dtype: object

In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Conv1D, GlobalMaxPooling1D, Dense, Dropout

# Load Twitter data
data = pd.read_csv('/content/drive/Shareddrives/Text mining Project/PROJECT/TWEET 2.csv')

# Preprocess data
X = data['text'].values
y = LabelEncoder().fit_transform(data['label'])

# Tokenize and pad sequences
max_words = 10000
max_len = 100
tokenizer = Tokenizer(num_words=max_words)
tokenizer.fit_on_texts(X)
X_seq = tokenizer.texts_to_sequences(X)
X_padded = pad_sequences(X_seq, maxlen=max_len)

# Load pre-trained GloVe embeddings
embedding_dim = 100
embedding_index = {}
with open('glove.twitter.27B.100d.txt', encoding='utf-8') as f:
    for line in f:
        values = line.split()
        word = values[0]
        coefs = np.asarray(values[1:], dtype='float32')
        embedding_index[word] = coefs

# Create an embedding matrix
embedding_matrix = np.zeros((max_words, embedding_dim))
for word, i in tokenizer.word_index.items():
    if i < max_words:
        embedding_vector = embedding_index.get(word)
        if embedding_vector is not None:
            # Ensure the dimensions match
            embedding_matrix[i] = embedding_vector[:embedding_dim]
# Build DCNN model with GloVe embeddings
model = Sequential()
model.add(Embedding(input_dim=max_words, output_dim=embedding_dim, input_length=max_len, weights=[embedding_matrix], trainable=False))
model.add(Conv1D(300, 1, activation='relu'))
model.add(Conv1D(300, 2, activation='relu'))
model.add(Conv1D(300, 3, activation='relu'))
model.add(Conv1D(300, 4, activation='relu'))
model.add(GlobalMaxPooling1D())
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Print model architecture
model.summary()

# Train the model
model.fit(X_padded, y, epochs=5, batch_size=32, validation_split=0.2)

# Evaluate the model
y_pred = (model.predict(X_padded) > 0.5).astype(int)
print(classification_report(y, y_pred))


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 100, 100)          1000000   
                                                                 
 conv1d (Conv1D)             (None, 100, 300)          30300     
                                                                 
 conv1d_1 (Conv1D)           (None, 99, 300)           180300    
                                                                 
 conv1d_2 (Conv1D)           (None, 97, 300)           270300    
                                                                 
 conv1d_3 (Conv1D)           (None, 94, 300)           360300    
                                                                 
 global_max_pooling1d (Glob  (None, 300)               0         
 alMaxPooling1D)                                                 
                                                        

In [ ]:
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix

# Print classification report
print(classification_report(y, y_pred))

# Additional metrics
accuracy = accuracy_score(y, y_pred)
conf_matrix = confusion_matrix(y, y_pred)

# Print additional metrics
print(f'Accuracy: {accuracy}')
print('Confusion Matrix:')
print(conf_matrix)

# Calculate and print sensitivity and specificity
true_negatives = conf_matrix[0, 0]
false_negatives = conf_matrix[1, 0]
true_positives = conf_matrix[1, 1]
false_positives = conf_matrix[0, 1]

sensitivity = true_positives / (true_positives + false_negatives)
specificity = true_negatives / (true_negatives + false_positives)

print(f'Sensitivity: {sensitivity}')
print(f'Specificity: {specificity}')


              precision    recall  f1-score   support

           0       0.81      0.87      0.84     22124
           1       0.83      0.75      0.79     18499

    accuracy                           0.82     40623
   macro avg       0.82      0.81      0.81     40623
weighted avg       0.82      0.82      0.82     40623

Accuracy: 0.8167540555842749
Confusion Matrix:
[[19218  2906]
 [ 4538 13961]]
Sensitivity: 0.7546894426725769
Specificity: 0.8686494304827337


In [ ]:
new_tweets = ["I love spreading kindness and positivity! #SpreadLove",
    "This is the worst thing ever! I can't believe people are so hateful. #Angry",
    "Why are muslims always fighting with others? Dont they have anything better to do?",
    "I can't stand women who act like they know it all.",
    "This is unacceptable! How can anyone say such hurtful things? #Angry",
    "Gay teachers should not be allowed near children",
    "Why are immigrants stealing all our jobs and money",
    "It's important to address hate speech and work towards building a more inclusive society. #SocialJustice"]

# Tokenize and pad sequences for new tweets
new_sequences = tokenizer.texts_to_sequences(new_tweets)
new_padded = pad_sequences(new_sequences, maxlen=max_len)

# Predict sentiment for new tweets
new_predictions = (model.predict(new_padded) > 0.5).astype(int)

# Print the results
for tweet, prediction in zip(new_tweets, new_predictions):
    sentiment = "Positive" if prediction == 1 else "Negative"
    print(f'Tweet: "{tweet}" - Sentiment: {sentiment}')


1/1 [==============================] - 0s 81ms/step
Tweet: "I love spreading kindness and positivity! #SpreadLove" - Sentiment: Positive
Tweet: "This is the worst thing ever! I can't believe people are so hateful. #Angry" - Sentiment: Positive
Tweet: "Why are muslims always fighting with others? Dont they have anything better to do?" - Sentiment: Negative
Tweet: "I can't stand women who act like they know it all." - Sentiment: Negative
Tweet: "This is unacceptable! How can anyone say such hurtful things? #Angry" - Sentiment: Positive
Tweet: "Gay teachers should not be allowed near children" - Sentiment: Negative
Tweet: "Why are immigrants stealing all our jobs and money" - Sentiment: Negative
Tweet: "It's important to address hate speech and work towards building a more inclusive society. #SocialJustice" - Sentiment: Positive
